# 1. Producing the data
In this task, we will implement Apache Kafka producers to simulate real-time data streaming. Spark is not allowed in this part since it’s simulating a streaming data source.  

1.	Your program should send one batch of browsing behaviour data every 5 seconds. One batch consists of a random 500-1000 rows from the browsing behaviour dataset. The CSV shouldn’t be loaded to memory at once to conserve memory (i.e. Read row as needed). Keep track of the start and end event_time. (You can assume the dataset is sorted by event_time.)  
2.	Add an integer column named ‘ts’ for each row, a Unix timestamp in seconds since the epoch. Spead your batch out evenly for 5 seconds.  
a.	For example, if you send a batch of 600 records at 2023-09-01 00:00:00 (ISO format: YYYY-MM-DD HH:MM:SS) -> (ts = 1693526400):  
-	Record 1-120: ts = 1693526400   
-	Record 121-240: ts = 1693526401   
-	Record 241-360: ts = 1693526402  
-	….  
3.	Read the transactions between the start and end event_time in 1.1 every 5 seconds (the same frequency as browsing behaviour) and create a batch.  
4.	Send your two batches from 1.1 and 1.3 to Kafka topics with an appropriate name.  
Note 1: In 1.1, “random 500-1000” means the number of rows is random, and the data file is still read sequentially.  
Note 2: All the data except for the ‘ts’ column should be sent in the original String type without changing to any other type. This is because we are simulating a streaming access log and need to reduce the required processing at the source.


In [1]:
from time import sleep
from kafka3 import KafkaProducer
import random
import datetime as dt
import csv
import os
import time
import csv
from json import dumps
from kafka3 import KafkaProducer
from time import sleep
from datetime import datetime
print(os.getcwd())

/home/student/BIGDATA/LABORATORY/ASSIGNMENTS/A2_B


In [2]:
curr_dir = os.getcwd()

In [3]:
# Configuring host and topic names for Kafka
hostip = "kafka"  # Hostname or IP for Kafka (e.g., Docker container)
browsing_behaviour_topic = 'BrowsingBehaviour-Topic'  # Topic for browsing events
transaction_topic = 'Transaction-Topic'  # Topic for transaction events

# Get the current working directory
curr_dir = os.getcwd()

# Defining paths to CSV files with datasets
browsing_csv_file_path = curr_dir + '/dataset/new_browsing_behaviour.csv'  # Browsing behaviour data file
transaction_csv_file_path = curr_dir + '/dataset/new_transactions.csv'  # Transactions data file


In [4]:
def publish_message(producer_instance, topic_name, key, value):
    """Publish a message to a Kafka topic."""
    try:
        # Convert key and value to bytes (Kafka requires bytes format)
        key_bytes = bytes(key, encoding='utf-8')
        value_bytes = bytes(value, encoding='utf-8')

        # Send the message to the specified topic
        producer_instance.send(topic_name, key=key_bytes, value=value_bytes)

        # Ensure the message is sent immediately
        producer_instance.flush()
        
        print(f'Message published to {topic_name} successfully. Data: {value}')

    except Exception as ex:
        # Handle any exceptions during publishing
        print('Exception in publishing message.')
        print(str(ex))


In [5]:
def connect_kafka_producer():
    """Connect to the Kafka producer."""
    _producer = None
    try:
        # Attempt to create a Kafka producer instance
        _producer = KafkaProducer(
            bootstrap_servers=[f'{hostip}:9092'], 
            api_version=(0, 10)
        )
    except Exception as ex:
        # Handle connection issues
        print('Exception while connecting to Kafka.')
        print(str(ex))
    finally:
        # Return the producer instance (even if None)
        return _producer

def get_random_batch(file_path, batch_size):
    """Read a random batch of rows from the CSV file."""
    batch = []  # Store the batch of rows
    start_event_time = None  # First event time in batch
    end_event_time = None  # Last event time in batch

    # Open the CSV and use DictReader to read rows
    with open(file_path, mode='r') as file:
        reader = csv.DictReader(file)

        # Randomly decide where to start reading
        start_index = random.randint(0, 1000)

        # Loop through rows and collect the batch
        for idx, row in enumerate(reader):
            if start_index <= idx < start_index + batch_size:
                batch.append(row)

                # Track the first event time
                if start_event_time is None:
                    start_event_time = row['event_time']
                
                # Update the last event time
                end_event_time = row['event_time']

            # Stop if we have enough rows
            if len(batch) == batch_size:
                break

    # Return the batch and event time range
    return batch, start_event_time, end_event_time


In [6]:
def get_transactions_between_time(file_path, start_time, end_time):
    """Get transactions that occurred between the given start and end times."""
    batch = []  # Store the matching transactions

    # Open the CSV file and read it with DictReader
    with open(file_path, mode='r') as file:
        reader = csv.DictReader(file)

        # Loop through each row in the file
        for row in reader:
            # Convert 'created_at' from string to datetime for comparison
            transaction_time = datetime.strptime(row['created_at'], '%Y-%m-%d %H:%M:%S.%f')

            # Check if the transaction falls within the given time range
            if start_time <= transaction_time <= end_time:
                batch.append(row)  # Add matching rows to the batch

    return batch  # Return the collected batch of transactions


In [7]:
def add_timestamp_to_batch(batch, start_ts):
    """Add a 'ts' column to each row in the batch."""
    batch_size = len(batch)
    interval = 5 / batch_size  # Spread timestamps evenly over the batch
    for idx, record in enumerate(batch):
        record['ts'] = int(start_ts + idx * interval)  # Calculate 'ts' for each record
    return batch  # Return the batch with added timestamps

if __name__ == '__main__':
    print('Publishing records...')  # Inform that publishing has started
    producer = connect_kafka_producer()  # Connect to Kafka producer
    
    if not producer:
        print("Error: Kafka producer connection failed.")  # Handle failed Kafka connection
    else:
        while True:
            batch_size = random.randint(500, 1000)  # Random batch size between 500 and 1000
            start_ts = int(time.time())  # Get the current Unix timestamp

            # Get a random batch from the browsing behaviour CSV
            browsing_batch, start_event_time, end_event_time = get_random_batch(browsing_csv_file_path, batch_size)
            
            if browsing_batch:
                browsing_batch_with_ts = add_timestamp_to_batch(browsing_batch, start_ts)  # Add timestamps
                for record in browsing_batch_with_ts:
                    record_str = dumps(record)  # Convert record to JSON
                    publish_message(producer, browsing_behaviour_topic, 'parsed', record_str)
                    sleep(0.01)  # Simulate real-time streaming

            # Convert event times to datetime format for filtering transactions
            start_event_time_dt = datetime.strptime(start_event_time, '%Y-%m-%d %H:%M:%S.%f')
            end_event_time_dt = datetime.strptime(end_event_time, '%Y-%m-%d %H:%M:%S.%f')

            # Get the transactions between the event times
            transaction_batch = get_transactions_between_time(transaction_csv_file_path, start_event_time_dt, end_event_time_dt)
            
            if transaction_batch:
                transaction_batch_with_ts = add_timestamp_to_batch(transaction_batch, start_ts)  # Add timestamps
                for record in transaction_batch_with_ts:
                    record_str = dumps(record)  # Convert record to JSON
                    publish_message(producer, transaction_topic, 'parsed', record_str)
                    sleep(0.01)  # Simulate real-time streaming

            # Pause for 5 seconds before sending the next batch
            sleep(5)


Publishing records...
Message published to BrowsingBehaviour-Topic successfully. Data: {"session_id": "fd4e6566-29a4-4e4b-94eb-e19ca996ec0a", "event_type": "VI", "event_time": "2024-01-01 00:44:40.406", "traffic_source": "MOBILE", "device_type": "iOS", "customer_id": "99970", "ts": 1728811857}
Message published to BrowsingBehaviour-Topic successfully. Data: {"session_id": "1b3d987c-5f63-4c48-b8fc-2524b12a2729", "event_type": "VI", "event_time": "2024-01-01 00:44:40.913", "traffic_source": "MOBILE", "device_type": "Android", "customer_id": "48198", "ts": 1728811857}
Message published to BrowsingBehaviour-Topic successfully. Data: {"session_id": "77a9189f-0fb4-4a48-9446-f346542f9d15", "event_type": "SER", "event_time": "2024-01-01 00:44:44.512", "traffic_source": "MOBILE", "device_type": "Android", "customer_id": "92931", "ts": 1728811857}
Message published to BrowsingBehaviour-Topic successfully. Data: {"session_id": "c843cfab-0203-46ea-9676-7b80092f5e63", "event_type": "VI", "event_tim

Message published to BrowsingBehaviour-Topic successfully. Data: {"session_id": "c1d823e4-616e-4d5b-bef0-fc46e7363e54", "event_type": "CL", "event_time": "2024-01-01 00:46:43.849", "traffic_source": "MOBILE", "device_type": "Android", "customer_id": "17821", "ts": 1728811857}
Message published to BrowsingBehaviour-Topic successfully. Data: {"session_id": "78dbfb26-8e0e-46b5-bba6-a889bdbf3529", "event_type": "CL", "event_time": "2024-01-01 00:46:44.517", "traffic_source": "", "device_type": "Android", "customer_id": "49646", "ts": 1728811857}
Message published to BrowsingBehaviour-Topic successfully. Data: {"session_id": "40522c3b-a797-4255-a145-d7bd3c23566c", "event_type": "SER", "event_time": "2024-01-01 00:46:45.304", "traffic_source": "MOBILE", "device_type": "Android", "customer_id": "91074", "ts": 1728811857}
Message published to BrowsingBehaviour-Topic successfully. Data: {"session_id": "78dbfb26-8e0e-46b5-bba6-a889bdbf3529", "event_type": "VC", "event_time": "2024-01-01 00:46:45

Message published to BrowsingBehaviour-Topic successfully. Data: {"session_id": "0c7ddb1d-fe2b-452c-90f2-90e666378e05", "event_type": "CL", "event_time": "2024-01-01 00:51:24.050", "traffic_source": "MOBILE", "device_type": "iOS", "customer_id": "59378", "ts": 1728811857}
Message published to BrowsingBehaviour-Topic successfully. Data: {"session_id": "2c070d8b-8543-4457-b70d-7ee9ff69070b", "event_type": "ATC", "event_time": "2024-01-01 00:51:26.073", "traffic_source": "WEB", "device_type": "iOS", "customer_id": "71359", "ts": 1728811857}
Message published to BrowsingBehaviour-Topic successfully. Data: {"session_id": "019c255d-da1f-40f0-b770-ebace212dff5", "event_type": "VP", "event_time": "2024-01-01 00:51:29.011", "traffic_source": "WEB", "device_type": "iOS", "customer_id": "91113", "ts": 1728811857}
Message published to BrowsingBehaviour-Topic successfully. Data: {"session_id": "418b67ff-7675-40ff-b04f-c521ee7bdb1c", "event_type": "VP", "event_time": "2024-01-01 00:51:36.550", "traf

Message published to BrowsingBehaviour-Topic successfully. Data: {"session_id": "674eb736-b7a9-4716-88f7-a8f7cee408e5", "event_type": "AP", "event_time": "2024-01-01 00:55:29.645", "traffic_source": "MOBILE", "device_type": "Android", "customer_id": "79526", "ts": 1728811858}
Message published to BrowsingBehaviour-Topic successfully. Data: {"session_id": "29e22221-b33d-414c-bdfd-f6754ad8fe2b", "event_type": "CL", "event_time": "2024-01-01 00:55:39.270", "traffic_source": "MOBILE", "device_type": "iOS", "customer_id": "42524", "ts": 1728811858}
Message published to BrowsingBehaviour-Topic successfully. Data: {"session_id": "3eb5fc80-69bd-4919-a11d-d2f772b54bb9", "event_type": "HP", "event_time": "2024-01-01 00:55:51.651", "traffic_source": "MOBILE", "device_type": "iOS", "customer_id": "52115", "ts": 1728811858}
Message published to BrowsingBehaviour-Topic successfully. Data: {"session_id": "c965389a-7899-45d8-86a3-eda2dfbe9414", "event_type": "HP", "event_time": "2024-01-01 00:55:54.12

Message published to BrowsingBehaviour-Topic successfully. Data: {"session_id": "29cab3eb-17b3-472a-8da7-7f5db672f60a", "event_type": "CL", "event_time": "2024-01-01 00:57:33.704", "traffic_source": "WEB", "device_type": "Android", "customer_id": "12525", "ts": 1728811858}
Message published to BrowsingBehaviour-Topic successfully. Data: {"session_id": "77e933cb-1e7f-4f62-a691-ae8764287b22", "event_type": "CL", "event_time": "2024-01-01 00:57:35.538", "traffic_source": "", "device_type": "Android", "customer_id": "45254", "ts": 1728811858}
Message published to BrowsingBehaviour-Topic successfully. Data: {"session_id": "77e933cb-1e7f-4f62-a691-ae8764287b22", "event_type": "VC", "event_time": "2024-01-01 00:57:36.944", "traffic_source": "", "device_type": "Android", "customer_id": "45254", "ts": 1728811858}
Message published to BrowsingBehaviour-Topic successfully. Data: {"session_id": "77e933cb-1e7f-4f62-a691-ae8764287b22", "event_type": "HP", "event_time": "2024-01-01 00:57:37.933", "tr

...

Message published to BrowsingBehaviour-Topic successfully. Data: {"session_id": "96c49377-635e-4c63-bc9c-9ef2cd57124b", "event_type": "CL", "event_time": "2024-01-01 01:36:35.846", "traffic_source": "MOBILE", "device_type": "iOS", "customer_id": "46805", "ts": 1728811898}
Message published to BrowsingBehaviour-Topic successfully. Data: {"session_id": "5dbb5657-ccd3-4c6d-9df1-393be9a14c8a", "event_type": "ATC", "event_time": "2024-01-01 01:36:37.006", "traffic_source": "MOBILE", "device_type": "iOS", "customer_id": "64894", "ts": 1728811898}
Message published to BrowsingBehaviour-Topic successfully. Data: {"session_id": "334e0663-1093-4fba-adad-e580cd1629ff", "event_type": "AP", "event_time": "2024-01-01 01:36:41.092", "traffic_source": "MOBILE", "device_type": "Android", "customer_id": "10434", "ts": 1728811898}
Message published to BrowsingBehaviour-Topic successfully. Data: {"session_id": "d05e66ab-e702-453c-996f-900b9369441c", "event_type": "VI", "event_time": "2024-01-01 01:36:43.8

Message published to BrowsingBehaviour-Topic successfully. Data: {"session_id": "d8bfc7c4-a523-4810-9e2d-54b33ae1e6c9", "event_type": "VI", "event_time": "2024-01-01 01:40:02.790", "traffic_source": "MOBILE", "device_type": "Android", "customer_id": "64599", "ts": 1728811899}
Message published to BrowsingBehaviour-Topic successfully. Data: {"session_id": "58a0f828-edbd-472b-82c8-afb807b7a175", "event_type": "SCR", "event_time": "2024-01-01 01:40:03.521", "traffic_source": "MOBILE", "device_type": "iOS", "customer_id": "31035", "ts": 1728811899}
Message published to BrowsingBehaviour-Topic successfully. Data: {"session_id": "dea7e754-00aa-4ed8-a26c-035e68f155fd", "event_type": "SER", "event_time": "2024-01-01 01:40:07.262", "traffic_source": "MOBILE", "device_type": "iOS", "customer_id": "88413", "ts": 1728811899}
Message published to BrowsingBehaviour-Topic successfully. Data: {"session_id": "e442b913-d342-43af-8724-faae60c9108a", "event_type": "HP", "event_time": "2024-01-01 01:40:17.

Message published to BrowsingBehaviour-Topic successfully. Data: {"session_id": "63e728fa-5a90-4364-b37c-54ff3df36cda", "event_type": "SER", "event_time": "2024-01-01 01:44:22.801", "traffic_source": "MOBILE", "device_type": "Android", "customer_id": "8036", "ts": 1728811899}
Message published to BrowsingBehaviour-Topic successfully. Data: {"session_id": "2ff0ff01-6fab-4ff7-8d6a-21713b7ee9de", "event_type": "HP", "event_time": "2024-01-01 01:44:39.088", "traffic_source": "MOBILE", "device_type": "Android", "customer_id": "70234", "ts": 1728811899}
Message published to BrowsingBehaviour-Topic successfully. Data: {"session_id": "584b4caf-b2e4-4af4-814d-33a8a80d37e9", "event_type": "VP", "event_time": "2024-01-01 01:44:42.329", "traffic_source": "MOBILE", "device_type": "iOS", "customer_id": "7322", "ts": 1728811899}
Message published to BrowsingBehaviour-Topic successfully. Data: {"session_id": "ec922d01-284b-4874-9ced-fa6c63185bc1", "event_type": "SCR", "event_time": "2024-01-01 01:44:4

Message published to BrowsingBehaviour-Topic successfully. Data: {"session_id": "6462adfd-a9a2-4a95-b29d-ebd005e339b0", "event_type": "CO", "event_time": "2024-01-01 01:47:57.383", "traffic_source": "MOBILE", "device_type": "Android", "customer_id": "46815", "ts": 1728811899}
Message published to BrowsingBehaviour-Topic successfully. Data: {"session_id": "0a5df247-a78c-4bc4-9f26-db6bd986124e", "event_type": "CL", "event_time": "2024-01-01 01:48:20.949", "traffic_source": "MOBILE", "device_type": "Android", "customer_id": "1109", "ts": 1728811899}
Message published to BrowsingBehaviour-Topic successfully. Data: {"session_id": "e9bc9df6-ab2c-4fda-bf1e-be74c750fc9f", "event_type": "HP", "event_time": "2024-01-01 01:48:39.830", "traffic_source": "MOBILE", "device_type": "iOS", "customer_id": "89367", "ts": 1728811899}
Message published to BrowsingBehaviour-Topic successfully. Data: {"session_id": "7c53b99e-9729-4bd1-9d19-19b35873af29", "event_type": "SCR", "event_time": "2024-01-01 01:48:5

KeyboardInterrupt: 